# Views, CTEs, and Joins

This file contains several exercises on views, common table expressions (CTEs), and joins. These topics are a step more involved and confusing than in the previous two files. Before attempting these problems, you'll probably need to do some research into these topics. Here are a few tutorials:

* [Tutorial on Views](https://www.sqlitetutorial.net/sqlite-create-view/)
* [Tutorial on CTEs](https://www.essentialsql.com/introduction-common-table-expressions-ctes/)
* [Tutorial on Joins](https://www.sqlitetutorial.net/sqlite-join/)

Here are a few other important notes I'd like you read before beginning:
* Some of these problems can get pretty involved. Queries with adequate spacing can go longer than 15 lines in some problems.
* Make sure you read each question thoroughly.
* Don't skip problems, as some problems may rely on previous problems being done correctly.
* Make sure you are saving your answers as you go, as some answers will simply be reworkings of previous answers.
* Some problems in the `Views` section involve you making persistent changes to the database itself. Specifically, if you create a view, you cannot create another view with the same name if it already exists. You may want to learn the `DROP VIEW` command. Or maybe a little easier,  you can recreate the entire database by simply running the `seed.py` script provided in this repo. If you're unsure, you can see your list of tables and views using the `.tables` dot-command from the `sqlite3` interface.

## Views
62) Look at the `yum` table. It is the stock data for Yum! Brands, Inc. from 2015 through 2019. Yum! is the company that owns Taco Bell, the best restaurant.
63) Query the `yum` table, aggregating by **both** month and year, with the following resulting columns:
* Year (4 digits)
* Month
* Average open, high, low, and close
* Total volume
Finally, sort this data so it's in proper chronological order.
64) Save the results of the previous query as a view named `yum_by_month`.
65) Create a view of `transactions` consisting of only three columns: year, month, and total sales in that month. Call this view `trans_by_month`.
66) Create a view of `transactions` consisting of only two columns: `employee_id` and the total sales corresponding to that employee. Call this view `trans_by_employee`.

## Common Table Expressions (CTEs)
CTEs are a convenient way of shortening SQL queries to keep your code DRY (**d**on't **r**epeat **y**ourself). You'll notice they're essentially the same in terms of the tasks they can accomplish, however CTEs are _temporary_. They vanish after the query has been called. Essentially, CTEs are single-use views.

Therefore, CTEs aren't needed to solve any of the following problems. You could use a view instead, however that would be wasteful since you'll never use them again. Additionally, for some problems, neither a view nor a CTE is truly needed, but the query would be very messy without one.

67) What's the most common first initial for pets in the `pets` table?
    * _Hint:_ Create a CTE that is simply the lowercased first letter of the pet's name. The solution is a simple `GROUP BY` from this CTE.
    * _Hint 2:_ You'll need the `SUBSTR()` and `LOWER()` functions.
68) Create taglines for each employee in the `employees` table. As a template, the first row of the result should look like this:

```
Christine Thompson started in 2005 and makes $123,696 working in sales. 
```

To do this easily, make a CTE featuring name (firstname + " " + lastname), job, salary (formatted), and year. Job title should be lowercased, _unless_ it is IT, in which case leave it capitalized. The solution is simple string concatenation off of this long CTE.

69) How many of our sales come from companies ending in each of "LLC", "Inc", "Ltd", or "PLC"? In a CTE, create a `company_type` column of values `"LLC"`, `"Inc"`, `"Ltd"`, `"PLC"`, or `"Other"`. Outside the CTE, find the total revenue from these categories, as well as their respective counts.
* _Hint:_ You'll need the `INSTR()` function.

## Joins
No, we're not done talking about views and CTEs! We're just going to start intermingling them in with further examples on joins, where the real power of these techniques becomes clearer.

70) Which employee made which sale? Join the `employees` table onto the `transactions` table by `employee_id`. You only need to include the employee's first/last name from `employees`.
71) What is the name of the employee who made the most in sales? Find this answer by doing a join as in the previous problem. Your resulting query will be difficult for someone else to read.
72) Solve the previous problem by joining `employees` onto the `trans_by_employee` view you made earlier.
73) Solve the previous problem by joining `employees` onto a CTE.
74) Next, the company will try to give bonuses based on performance. Show all employees who've made more in sales than 1.5 times their salary. (You may use whatever technique you'd like to do the join: view, CTE, or even a subquery!)
75) Do we have potentially erroneous rows? Find all transactions which occurred _before_ the employee was even hired! (Make sure each transaction only occupies one row).
76) Among all transactions that occurred from 2015 to 2019, create a table that is the monthly revenue of our company versus the total trading volume of Yum! in that month. Format the columns nicely. That is, a sample row of your result might look like this:

```
| year | month | company_revenue | yum_trade_volume |
|------|-------|-----------------|------------------|
| 2017 |    03 |        $100,000 |      125,000,000 |
```

* _Hint:_ You don't need any `WHERE` statements here. You can get the right answer simply by changing what kind of join you do!

77) Repeat the previous problem, but in addition to the total volume, include:
* The lowest price that month (ie, lowest low)
* The highest price that month (ie, highest high)


In [1]:
# Import Pandas and Create_Engine

from sqlalchemy import create_engine
import pandas as pd
import sqlite3

con = sqlite3.connect("ladder.db")

62) Look at the yum table. It is the stock data for Yum! Brands, Inc. from 2015 through 2019. Yum! is the company that owns Taco Bell, the best restaurant. 

In [2]:
sql = '''
SELECT *
FROM yum
LIMIT 5;
'''
pd.read_sql_query(sql, con)

,date,open,high,low,close,volume
0,2015-01-02,52.796551,52.846874,51.761322,52.012939,2283400
1,2015-01-05,52.056076,52.278934,50.948959,50.956146,4418600
2,2015-01-06,51.301224,51.329979,49.877785,50.330696,5004400
3,2015-01-07,51.020847,52.056076,50.884254,51.998562,4554100
4,2015-01-08,52.408340,53.040977,52.365204,52.904385,4258200


63) Query the yum table, aggregating by both month and year, with the following resulting columns:

Year (4 digits)
Month
Average open, high, low, and close
Total volume Finally, sort this data so it's in proper chronological order. 

In [3]:
sql = '''
SELECT strftime('%Y-%m', date) AS month_year, 
AVG(open) AS 'avg_open', AVG(high) AS 'avg_high',
AVG(close) AS 'avg_close', SUM(volume) AS 'total_volume'
FROM yum
GROUP BY month_year
ORDER BY month_year
LIMIT 10;
'''
pd.read_sql_query(sql, con)

,month_year,avg_open,avg_high,avg_close,total_volume
0,2015-01,52.221423,52.736520,52.192667,89074400
1,2015-02,54.183662,54.823868,54.534791,98621800
2,2015-03,56.845958,57.324685,56.818509,108827600
3,2015-04,58.533429,59.140050,58.740885,117743300
4,2015-05,65.739757,66.475916,65.755572,131485600
5,2015-06,65.550291,66.017907,65.579373,69412500
6,2015-07,64.009869,64.498399,63.883079,116786700
7,2015-08,60.153709,60.950327,60.029784,155692800
8,2015-09,57.495806,58.036356,57.506419,97615900
9,2015-10,52.663224,53.319391,52.669760,232475400


64) Save the results of the previous query as a view named yum_by_month

In [7]:
sql = '''
CREATE VIEW "yum_by_month" AS 
SELECT strftime('%Y-%m', date) AS month_year, 
AVG(open) AS 'avg_open', AVG(high) AS 'avg_high',
AVG(close) AS 'avg_close', SUM(volume) AS 'total_volume'
FROM yum
GROUP BY month_year
ORDER BY month_year
LIMIT 10;
'''


65) Create a view of transactions consisting of only three columns: year, month, and total sales in that month. Call this view trans_by_month

In [17]:
sql = '''
CREATE VIEW "trans_by_month" AS
SELECT STRFTIME('%Y', orderdate) AS 'year', 
STRFTIME('%m', orderdate) AS 'month', 
(quantity * unit_price) as 'total_sales'
FROM transactions
GROUP BY month
ORDER BY year DESC;
'''


66) Create a view of transactions consisting of only two columns: employee_id and the total sales corresponding to that employee. Call this view trans_by_employee.

In [51]:
sql = '''
CREATE VIEW "trans_by_employee" AS 
SELECT employee_id,(quantity * unit_price) as 'total_sales' 
FROM transactions
GROUP BY employee_id;
'''


67) What's the most common first initial for pets in the pets table? * Hint: Create a CTE that is simply the lowercased first letter of the pet's name. The solution is a simple GROUP BY from this CTE. * Hint 2: You'll need the SUBSTR() and LOWER() functions. 

In [24]:
sql = '''
WITH pet_first_letter AS (
  SELECT LOWER(SUBSTR(name, 1, 1)) AS first_letter
  FROM pets
)

SELECT first_letter, COUNT(*) AS count
FROM pet_first_letter
GROUP BY first_letter
ORDER BY count DESC;
'''
pd.read_sql_query(sql, con)

,first_letter,count
0,p,3
1,m,2
2,s,1
3,o,1
4,g,1
5,e,1
6,d,1
7,c,1
8,b,1
9,a,1


68) Create taglines for each employee in the employees table. As a template, the first row of the result should look like this:

Christine Thompson started in 2005 and makes $123,696 working in sales. 
To do this easily, make a CTE featuring name (firstname + " " + lastname), job, salary (formatted), and year. Job title should be lowercased, unless it is IT, in which case leave it capitalized. The solution is simple string concatenation off of this long CTE.

In [31]:
 sql = '''
WITH employee_info AS (
  SELECT 
    firstname || ' ' || lastname AS name,
    CASE 
      WHEN job = 'IT' THEN job
      ELSE LOWER(job)
    END AS job_title,
    '$' || FORMAT(salary, 0) AS formatted_salary,
    STRFTIME('%Y', startdate) AS hire_year
  FROM employees
)

SELECT name || ' started in ' || hire_year || ' and makes ' || formatted_salary || ' working in ' || job_title || '.'
FROM employee_info;
'''
pd.read_sql_query(sql, con)

,name || ' started in ' || hire_year || ' and makes ' || formatted_salary || ' working in ' || job_title || '.'
0,Christine Thompson started in 2005 and makes $...
1,Thomas Peck started in 2011 and makes $112972 ...
2,Christopher Robles started in 2003 and makes $...
3,Elizabeth Munoz started in 1993 and makes $558...
4,Janice Martin started in 2011 and makes $62007...
...,...
95,Tina Flynn started in 2006 and makes $55916 wo...
96,Allen Cross started in 2019 and makes $118571 ...
97,Brittany Maynard started in 2003 and makes $69...
98,Bradley Romero started in 2009 and makes $4526...


69) How many of our sales come from companies ending in each of "LLC", "Inc", "Ltd", or "PLC"? In a CTE, create a company_type column of values "LLC", "Inc", "Ltd", "PLC", or "Other". Outside the CTE, find the total revenue from these categories, as well as their respective counts.

Hint: You'll need the INSTR() function.

In [35]:
sql = '''
WITH company_types AS (
  SELECT 
    customer,
    CASE 
      WHEN INSTR(customer, ' LLC') > 0 THEN 'LLC'
      WHEN INSTR(customer, ' Inc') > 0 THEN 'Inc'
      WHEN INSTR(customer, ' Ltd') > 0 THEN 'Ltd'
      WHEN INSTR(customer, ' PLC') > 0 THEN 'PLC'
      ELSE 'Other'
    END AS company_type
  FROM transactions
)

SELECT 
  company_type, 
  COUNT(*) AS count, 
  SUM(unit_price * quantity) AS total_revenue 
FROM transactions 
JOIN company_types 
  ON transactions.customer = company_types.customer
GROUP BY company_type;
'''
pd.read_sql_query(sql, con)

,company_type,count,total_revenue
0,Inc,1318472,1.770680e+08
1,LLC,931347,1.274192e+08
2,Ltd,1889125,2.525594e+08
3,Other,19983406,2.740305e+09
4,PLC,1280219,1.775684e+08


70) Which employee made which sale? Join the `employees` table onto the `transactions` table by `employee_id`. You only need to include the employee's first/last name from `employees`.

In [46]:
sql = '''
SELECT 
  employees.firstname, 
  employees.lastname, 
  (transactions.unit_price * transactions.quantity) as 'sale'

FROM transactions 
JOIN employees 
ON transactions.employee_id = employees.ID;
'''
pd.read_sql_query(sql, con)

,firstname,lastname,sale
0,Christopher,Carlson,246.00
1,Christopher,Carlson,264.00
2,Christopher,Carlson,311.50
3,Christopher,Carlson,115.50
4,Christopher,Carlson,123.75
...,...,...,...
50076,Robert,Logan,110.00
50077,Robert,Logan,195.25
50078,Robert,Logan,227.50
50079,Robert,Logan,98.00


71) What is the name of the employee who made the most in sales? Find this answer by doing a join as in the previous problem. Your resulting query will be difficult for someone else to read.

In [48]:
sql = '''
SELECT 
  employees.firstname, 
  employees.lastname, 
  (transactions.unit_price * transactions.quantity) as 'sale'

FROM transactions 
JOIN employees 
ON transactions.employee_id = employees.ID
GROUP BY employees.ID
ORDER BY sale DESC;
'''
pd.read_sql_query(sql, con)

,firstname,lastname,sale
0,Lisa,Delgado,341.25
1,Aaron,Kelley,332.50
2,Allen,Cross,311.25
3,Nicholas,Pope,299.25
4,Thomas,Miller,267.00
...,...,...,...
63,William,Roth,33.75
64,Anna,Fischer,33.00
65,Anthony,Hogan,29.25
66,Caleb,Figueroa,21.00


72) Solve the previous problem by joining employees onto the trans_by_employee view you made earlier.

In [54]:
sql = '''
SELECT 
  employees.firstname, 
  employees.lastname, 
  trans_by_employee.
FROM employees 
JOIN trans_by_employee 
  ON employees.ID = trans_by_employee.employee_id;
'''
pd.read_sql_query(sql, con)

DatabaseError: Execution failed on sql '
SELECT 
  employees.firstname, 
  employees.lastname, 
  trans_by_employee.total_sales
FROM employees 
JOIN trans_by_employee 
  ON employees.ID = trans_by_employee.employee_id;
': no such column: trans_by_employee.total_sales

73) Solve the previous problem by joining employees onto a CTE.